# 案例10

In [ ]:

import os
from datetime import datetime
import numpy as np
import pandas as pd
import scipy as sc
from keyfactordiag.feature_selector import FeatureSelector
from tqdm import tqdm

In [ ]:
df_smtTrace = pd.read_excel('case/10/6_smt_trace.xlsx')


In [ ]:
filedfail = pd.read_excel('case/10/fieldfail_mbsn.xlsx')
filedfail['label']=1

**Preprocess**
1. 刪除LOCATION1為空值的數據
2. 刪掉同一個WIP_SN, 同一個LOCATION1會有多筆上料紀錄的location
    - 刪除前共xxxx個location
    - 刪除後剩下1165個location
3. 合併VENDER_CODE+LOT_CODE, 合併VENDER_CODE+DATE_CODE

In [ ]:
#1
df_smtTrace.dropna(subset=['LOCATION1'], inplace=True)

#2
location_drop=[]
for location in df_smtTrace['LOCATION1'].unique(): 
    df = df_smtTrace.query(f"LOCATION1=='{location}'")
    isduplicated = df.duplicated(subset=['WIP_SN','LOCATION1'])
    if isduplicated.any():
        location_drop.append(location)        
df_smtTrace= df_smtTrace[~df_smtTrace['LOCATION1'].isin(location_drop)]    
#3
df_smtTrace['LOT_CODE']=df_smtTrace['VENDER_CODE']+df_smtTrace['LOT_CODE']
df_smtTrace['DATE_CODE']=df_smtTrace['VENDER_CODE']+df_smtTrace['DATE_CODE']

**組大表**


In [ ]:
bigtable={}
'''
- 目的: 每一個序號的SMT TRACE過站資訊
- 作法: 
    1. filter location1 name
    3. label=1, 全都是fail
    5. 重新命名欄位名稱        
'''

col_test = ['SERIAL_NUMBER','IN_STATION_TIME','是否是不良?']
col_smttrace = ['WIP_SN','USED_PN','VENDER_CODE','LOT_CODE','DATE_CODE','LOCATION1']
col_output = ['WIP_SN','label','USED_PN','VENDER_CODE','LOT_CODE','DATE_CODE']
for location in tqdm(df_smtTrace['LOCATION1'].unique()): #['0UC2,0UC2,', '0PUG101,0PUG101,']:
#for location in ['0PUZ4,0PUZ2,0PUZ3,0PUZ4,0PUZ2,0PUZ3,', '0PUG101,0PUG101,']:
    #step 1 
    df_smttrace = df_smtTrace.query(f"LOCATION1=='{location}'")[col_smttrace]
    #step 2 
    #step 3
    df_smttrace['label']=1 #Fail
    #step 4 
    #step 5
    df_smttrace = df_smttrace[col_output]
    _col_rename = dict(zip(col_smttrace, [f'smttrace:{location}:{f}' if f not in ['WIP_SN','label'] else f for f in col_smttrace]))
    df_smttrace.rename(columns=_col_rename, inplace=True)
    bigtable[f'smt_trace:{location}'] = df_smttrace
    
df_bigtable = filedfail 
for k, data in tqdm(bigtable.items()):
    df_bigtable = pd.merge(df_bigtable, data, on=['WIP_SN','label'], how='left')
bigtable['big'] = df_bigtable    


**集中性分析**

In [ ]:
#====== Factor Ranking =======
def getentropy(df, feature, isonehot=True):
    if isonehot:
        data_f = df[df[feature]==1]
        len_f1=len(data_f)
        p_data1 = data_f['label'].value_counts() 
        ent1 = sc.stats.entropy(p_data1/len_f1, base=2)
        data_f = df[df[feature]==0]
        len_f0=len(data_f)
        p_data0 = data_f['label'].value_counts()
        ent0 = sc.stats.entropy(p_data0/len_f0, base=2)

        ent_mean = ent1*(len_f1/(len_f1+len_f0))+ent0*(len_f0/(len_f1+len_f0))
        p_data1.rename(index={1:'Fail',0:'Pass'}, inplace=True)
        p_data0.rename(index={1:'Fail',0:'Pass'}, inplace=True)
        return (ent1, p_data1, ent0, p_data0, ent_mean)
    else:
        entlist=[]
        for v in df[feature].unique():
            data_f = df[df[feature]==v]
            len_f=len(data_f)
            p_data = data_f['label'].value_counts() 
            ent_ = sc.stats.entropy(p_data/len_f, base=2)
            entlist.append(ent_*len_f/len(df))
        ent = np.sum(entlist)
        return ent

In [ ]:


df_bigtable_ = bigtable['big']
##FIXME 暫時移除時間和SN的欄位
newcols= list(filter(lambda x: 'TIME' not in x, df_bigtable_.columns))
newcols= list(filter(lambda x: 'KEY_PART_SN' not in x, newcols))
df_bigtable = df_bigtable_[newcols]
train_labels = df_bigtable['label'].copy()
train = df_bigtable.drop(columns = ['label'])
del train['WIP_SN']
fs = FeatureSelector(data = train, labels = train_labels)

#remove不能用的因子 (missing value>30%, single unique)
fs.identify_missing(missing_threshold=0.3)
fs.identify_single_unique()
train = fs.remove(methods = ['missing', 'single_unique'])
print(fs.check_removal())
fs = FeatureSelector(data = train, labels = train_labels)        

#One-Hot Encoding
fs.identify_zero_importance(task = 'classification', eval_metric = 'auc', n_iterations = 1, early_stopping = True)
one_hot_features = fs.one_hot_features
base_features = fs.base_features
print('There are %d original features' % len(base_features))
print('There are %d one-hot features' % len(one_hot_features))        

#Layer 1 factor analysis 
ocols=['feature','Qty{P}','Qty{N}','entropy(mean)']
df_entdata = pd.concat([fs.data_all[one_hot_features], train_labels], axis=1)
rootent = sc.stats.entropy(df_entdata['label'].value_counts()/len(df_entdata['label']), base=2)
print('root ent:',rootent)
entlist = []
for f in tqdm(df_entdata.columns):
    if f=='label':
        continue
    ent1, p_data1, ent0, p_data0, ent = getentropy(df_entdata, f)
    #entlist.append([f, ent1, str(dict(p_data1.sort_index(ascending=False))), ent0, str(dict(p_data0.sort_index(ascending=False))), ent])
    entlist.append([f, ent1, dict(p_data1.sort_index(ascending=False)), ent0, dict(p_data0.sort_index(ascending=False)), ent])

df_ent = pd.DataFrame(entlist)
df_ent.columns = ['feature','entropy(P)','Qty(P)','entropy(N)','Qty(N)','entropy(mean)']
#df_ent['exp'] = df_ent['entropy(mean)'].map(lambda x: (rootent-x)/rootent) #exp: 解釋了多少比例的不確定性
#df_ent = df_ent.sort_values(by='entropy(mean)').reset_index(drop=True)
#self.factortable['main']=df_ent


#--- output---
def getothers(x, cols):
    k=':'.join(x.split(':')[:3])
    ret = list(filter(lambda x: k in x, cols))
    ret = [c.split(':')[-1] for c in ret]
    return ret        

outputcols=['Type', 'Factor A', 'Factor B', 'Factor C', 'Pass', 'Fail','Failrate', 'Others','Others-Pass', 'Others-Fail', 'entropy(mean)','rank']

#Table格式 ##FIXME
df = df_ent.copy()
cols = df['feature']
df['Type']=df['feature'].map(lambda x: x.split(':')[0])
df['Factor A']=df['feature'].map(lambda x: x.split(':')[1])
df['Factor B']=df['feature'].map(lambda x: x.split(':')[2])
df['Factor C']=df['feature'].map(lambda x: x.split(':')[3])
df['Others'] = df['feature'].map(lambda x: getothers(x, cols))
del df['feature']
df['Pass']=df['Qty(P)'].map(lambda x: x.get('Pass',0))
df['Fail']=df['Qty(P)'].map(lambda x: x.get('Fail',0) )
df['Failrate']=df['Fail']/(df['Pass']+df['Fail'])
df['Failrate'] = df['Failrate'].map(lambda x: np.round(x,2))
df['Others-Pass']=df['Qty(N)'].map(lambda x: x.get('Pass',0))
df['Others-Fail']=df['Qty(N)'].map(lambda x: x.get('Fail',0) )
del df['Qty(P)']
del df['Qty(N)']
del df['entropy(P)']
del df['entropy(N)']
df.sort_values(by='Fail', ascending=False, inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index':'rank'}, inplace=True)            
df = df[outputcols]


writer=pd.ExcelWriter('output/case10_集中性分析Report.xlsx') 
df.to_excel(writer, 'main', index=False)
writer.save()